In [ ]:
#|default_exp utils

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Utils
> Various utils for cleaning, organizing and capturing other information.

## Generic utils

In [ ]:
#|export
import pickle
import numpy as np
from pathlib import Path
from fastcore.foundation import L
from fastcore.xtras import globtastic
from fastcore.meta import delegates
import pathlib
from fastcore.test import test_eq
from fastcore.script import call_parse

In [ ]:
#| hide
#| local
%cd ~

/home/deven


In [ ]:
#| exporti
def check_files(
    files, # files to validate
    ):
    flen = files.__len__()
    if flen <= 0:
        print(f'Found {flen} files')
        print(f'Check `path` and try again')
        return False
    
    if isinstance(files[0], str):
        _type =  'npy' if files[0].endswith('npy') else 'pkl'
    elif isinstance(files[0], Path):
        _type =  'npy' if files[0].name.endswith('npy') else 'pkl'
        
    print(f'Found {flen} {_type} files')
    print('-'*45)
    return True

In [ ]:
#|hide
#| local
files = globtastic('embeddings/A_Modest_Proposal/pkl', recursive=False)
_ = check_files(files)

Found 0 files
Check `path` and try again


In [ ]:
#|hide
#| local
files = globtastic('embeddings/A_Modest_Proposal/', recursive=False)
_ = check_files(files)

Found 0 files
Check `path` and try again


In [ ]:
#|export
def get_data(
    fname: str|Path # path to the file
    )->str: # returns content of the file
    "Reads from a txt file"
    with open(fname, 'r') as f:
        all_text = f.read()
    return all_text

In [ ]:
#|export
def load_pmi(
    fname: str|Path  # name of pmi file
) -> np.ndarray:  # pmi matrix
    """
    Loads the PMI matrix
    """
    file_ = loader(fname, '.npy')
    pmi = np.load(file_)
    print(f'Loaded {name}')
    return pmi

In [ ]:
#|export
@delegates(globtastic)
def loader(
    path: str|Path,  # path to a given folder,
    extension: str,  # extension of the file you want
    **kwargs
) -> L:  # returns `L`
    "Given a Path and an extension, returns all files with the extension in the path"
    files = globtastic(path, file_glob=f'*{extension}', **kwargs).map(Path)

    return files

In [ ]:
#|export
def load_dictionary(
    fname: str , # path to the pkl file
    )->dict: # returns the contents 
    """
    Given a fname, function loads a `pkl` dictionary
    from the current directory
    """
    fname = open(fname, 'rb')
    data = pickle.load(fname)
    return data

In [ ]:
#|export
def normalize(
    data: np.ndarray,  # input array
) -> np.ndarray:  # normalized array
    """
    Given an input array, return normalized array
    """
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
test_eq(normalize([1, 2, 3, 4, 5]), [0.  , 0.25, 0.5 , 0.75, 1.  ])

In [ ]:
#|export
@call_parse
def chelp():
    "Show help for all console scripts"
    from fastcore.xtras import console_help
    console_help('clean_plot')

In [ ]:
!cp_help

check_len                       Takes name of a txt file and writes the tokenized sentences into a new txt file
corr_hm                         Generates correlation plots from normalized SSMs
cp_help                         Show help for all console scripts
heatmaps                        Generates plots for embeddings in the folder
heatmaps_pkl                    Generates SSMs from pkl files
histograms                      Generates histograms for embeddings in the folder
lex_ts                          Generate lexical TS from Lexical SSM
make_pkl                        Create pkl for time series from embeddings
ts_pkl                          Plot timeseries from the pkl file


## Utils for cleaning text

In [ ]:
#| export
import re
from fastcore.script import call_parse

Before using any of the cleaning utils in the file, please run `download_nltk_dep` first.

In [ ]:
#|export
def download_nltk_dep():
    """
    Downloads the `nltk` dependencies
    """
    import nltk
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')
    nltk.download('omw-1.4')

In [ ]:
#| hide
download_nltk_dep()

[nltk_data] Downloading package punkt to /home/deven/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/deven/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/deven/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/deven/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/deven/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
#|export
def split_by_newline(
    text: str, # sentences separated by \n
    ) -> L: # list of sentences
    """
    Only use when sentences are already tokenized
    returns sentences split by `\n`
    """
    return L([line for line in text.split('\n') if len(line) > 0])

In [ ]:
text = "Hello there!\nThis is how this functions works!"
split_by_newline(text)

(#2) ['Hello there!','This is how this functions works!']

In [ ]:
#|export
def rm_useless_spaces(
    t: str, # sentence with extra spaces
    ) -> str: # sentence without extra spaces
    """
    Removes useless spaces
    """
    _re_space = re.compile(' {2,}')
    return _re_space.sub(' ', t).lstrip().rstrip()

In [ ]:
rm_useless_spaces('  This is      test sentence.  This removes  all the extra  spaces.  ')

'This is test sentence. This removes all the extra spaces.'

In [ ]:
#|export
def make_sentences(
    text: str, # bulk text
    ) -> L: # list of sentences
    """
    Converts given bulk into sentences
    """
    all_cleaned = text.replace('\n', ' ')
    all_cleaned = rm_useless_spaces(all_cleaned)
    all_cleaned = all_cleaned.strip()
    all_cleaned = unidecode.unidecode(all_cleaned)
    sentences = sent_tokenize(all_cleaned)
    return L(sentences)

In [ ]:
#|export
def write_to_file_cleaned(
    sentences: list, # list of sentences 
    fname: str, # name of output file
    ) -> None:
    """
    Writes the sentences to a .txt file
    """
    with open(f'{fname.stem}_cleaned.txt', 'w') as f:
        for line in sentences:
            f.write(f'{line}\n')
    f.close()

In [ ]:
#|export
@call_parse
def clean(
    fname: str, # name of input txt file
    ) -> None:
    "Takes name of a txt file and writes the tokenized sentences into a new txt file"
    fname = Path(fname)
    text = get_data(fname)
    sentences = make_sentences(text)
    print(f'{fname.name} contains {len(sentences)} sentences')
    write_to_file_cleaned(sentences, fname)

In [ ]:
#|export
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer

All functions mentioned above are merged into a single function called `clean`. 
You only need to give it the name of the .txt file that you want to clean and call the function

In [ ]:
#|hide
#| local
%cd /home/deven/projects/clean_plot/nbs

/home/deven/projects/clean_plot/nbs


In [ ]:
fname = '../files/dummy.txt'
text = get_data(fname)
print(text)

MARLEY was dead: to begin with. There is no doubt
whatever about that. The register of his burial was
signed by the clergyman, the clerk, the undertaker,
and the chief mourner. Scrooge signed it: and
Scrooge's name was good upon 'Change, for anything he
chose to put his hand to. Old Marley was as dead as a
door-nail.

Mind! I don't mean to say that I know, of my
own knowledge, what there is particularly dead about
a door-nail. I might have been inclined, myself, to
regard a coffin-nail as the deadest piece of ironmongery
in the trade. But the wisdom of our ancestors
is in the simile; and my unhallowed hands
shall not disturb it, or the Country's done for. You
will therefore permit me to repeat, emphatically, that
Marley was as dead as a door-nail.

This is a new sentence.


It goes from this to 

In [ ]:
#| export
import unidecode

In [ ]:
make_sentences(get_data(fname))

(#11) ['MARLEY was dead: to begin with.','There is no doubt whatever about that.','The register of his burial was signed by the clergyman, the clerk, the undertaker, and the chief mourner.',"Scrooge signed it: and Scrooge's name was good upon 'Change, for anything he chose to put his hand to.",'Old Marley was as dead as a door-nail.','Mind!',"I don't mean to say that I know, of my own knowledge, what there is particularly dead about a door-nail.",'I might have been inclined, myself, to regard a coffin-nail as the deadest piece of ironmongery in the trade.',"But the wisdom of our ancestors is in the simile; and my unhallowed hands shall not disturb it, or the Country's done for.",'You will therefore permit me to repeat, emphatically, that Marley was as dead as a door-nail.'...]

The `clean` function writes these sentences into a txt file with the name `<fname>_cleaned.txt` 

In [ ]:
#|export
def get_wordnet_pos(
    word: str, # input word token
    ) -> str: # POS of the given word
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
#| export
from nltk.corpus import stopwords

In [ ]:
#|export
def remove_stopwords(
    sentence: str, # input sentence
    ) -> str: # output sentence
    """
    Takes a sentence and removes stopwords from it
    """
    sentences = []
    STOPWORDS = set(stopwords.words('english'))
    for word in sentence.split():
        if word.lower() not in STOPWORDS:
            sentences.append(word)
    return ' '.join(sentences)

In [ ]:
#|export
def remove_punctuations(
    sentence: str, # input sentence
    ) -> str: # output sentence
    """
    Takes a sentence and removes punctuations from it
    """
    pat2 = re.compile('[^a-zA-Z0-9 ]+')
    pat1 = re.compile('[\s]+')

    doc = pat2.sub(' ', sentence)
    doc = pat1.sub(' ', doc)
    doc = doc.strip()
    return doc

In [ ]:
#|export
def remove_punc_clean(
    sentence: str, # input sentence
    lemmatize: bool = False, # flag to `lemmatize`
    ) -> str:
    """
    Takes a sentence and removes punctuations and stopwords from it
    
    Will lemmatize words if `lemmatize = True`
    """
    doc = remove_punctuations(sentence)
    doc = remove_stopwords(doc)
    
    
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        doc = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in doc.split()])
    return doc

:::{.callout-note}

It is possible that while using `remove_punc_clean`, a sentence might get eliminated completely as it only contained stopwords.

:::

In [ ]:
#|export
def process_for_lexical(
    fname: str, # name of the input txt file
    ) -> L: # 
    "Given an input txt file, return removed sentences"
    fname = Path(fname)
    all_data = get_data(fname)
    sentences = make_sentences(all_data)
    clean_sentences = []
    removed_sentences = []
    for i, sentence in enumerate(sentences):
        t = remove_punc_clean(sentence)
        if len(t) > 0:
            clean_sentences.append(t)
        else:
            removed_sentences.append(i)

    # write_to_file_lexical(clean_sentences, fname)
    print('Done processing', fname.name)
    return L(removed_sentences)

### Example contd.

In [ ]:
data = get_data(fname)
sentences = make_sentences(data)
sentences

(#11) ['MARLEY was dead: to begin with.','There is no doubt whatever about that.','The register of his burial was signed by the clergyman, the clerk, the undertaker, and the chief mourner.',"Scrooge signed it: and Scrooge's name was good upon 'Change, for anything he chose to put his hand to.",'Old Marley was as dead as a door-nail.','Mind!',"I don't mean to say that I know, of my own knowledge, what there is particularly dead about a door-nail.",'I might have been inclined, myself, to regard a coffin-nail as the deadest piece of ironmongery in the trade.',"But the wisdom of our ancestors is in the simile; and my unhallowed hands shall not disturb it, or the Country's done for.",'You will therefore permit me to repeat, emphatically, that Marley was as dead as a door-nail.'...]

Let's continue the same example from above

Here, the `remove_punc_clean` function removes punctuations, STOPWORDS and lemmatizes the word and returns the cleaned sentence. 

:::{.callout-Note}

It is possible that a sentence may be removed completely as it may contain only STOPWORDS.

:::
This method is to be used for methods involving lexical analysis.

Without lemmatization

In [ ]:
for sentence in sentences:
    print(remove_punc_clean(sentence))

MARLEY dead begin
doubt whatever
register burial signed clergyman clerk undertaker chief mourner
Scrooge signed Scrooge name good upon Change anything chose put hand
Old Marley dead door nail
Mind
mean say know knowledge particularly dead door nail
might inclined regard coffin nail deadest piece ironmongery trade
wisdom ancestors simile unhallowed hands shall disturb Country done
therefore permit repeat emphatically Marley dead door nail
new sentence


With Lemmatization

In [ ]:
for sentence in sentences:
    print(remove_punc_clean(sentence, lemmatize=True))

MARLEY dead begin
doubt whatever
register burial sign clergyman clerk undertaker chief mourner
Scrooge sign Scrooge name good upon Change anything chose put hand
Old Marley dead door nail
Mind
mean say know knowledge particularly dead door nail
might inclined regard coffin nail deadest piece ironmongery trade
wisdom ancestor simile unhallowed hand shall disturb Country do
therefore permit repeat emphatically Marley dead door nail
new sentence


In [ ]:
clean('../files/dummy.txt')

dummy.txt contains 11 sentences


In [ ]:
process_for_lexical('../files/dummy.txt')

Done processing dummy.txt


(#0) []

In [ ]:
Path('../files/').ls()

(#2) [Path('../files/dummy.txt'),Path('../files/dummy_cleaned.txt')]

In [ ]:
#| export
def num_words(
    sentence: str, # input sentence
    )->int: # number of words
    "Returns the number of words in a sentence"
    return len(remove_punctuations(sentence).split())

In [ ]:
print(sentences[0])
num_words(sentences[0])

MARLEY was dead: to begin with.


6

In [ ]:
print(sentences[1])
num_words(sentences[1])

There is no doubt whatever about that.


7